In [1]:
using ITensors, TensorKit, TensorOperations

In [41]:
function HubbardDMRG(t::Float64, U::Float64, μ::Float64, L::Int)

    sites = siteinds("Qudit",L; dim=L,  conserve_qns=true, conserve_number=true)

    H = OpSum()
    for i in (1:L-1)
        H += -μ - U/2, "n", i
        H += U/2, "n", i, "n", i
        H += -t, "a", i, "adag", i+1
        H += -t, "adag", i, "a", i+1
    end
    # Add the on-site terms that couldn't be added in the loop (due to i+1)
    H += -μ - U/2, "n", L
    H += U/2, "n", L, "n", L

    # Add periodic boundary conditions.
    H += -t, "a", 1, "adag", L
    H += -t, "adag", 1, "a", L
    
    Ham = MPO(H, sites)

    state = [isodd(n) ? "1" : "1" for n=1:L]
    psi0 = randomMPS(sites,state)

    nsweeps = 10
    maxdim = [40, 80, 400, 400, 800, 800, 1000, 1000, 1000, 1000]
    cutoff = [1e-12]
  
    energy, psi = dmrg(Ham,psi0; nsweeps, maxdim, cutoff)
    N = expect(psi, "n")
    @show flux(psi)

    return energy, N
end

HubbardDMRG (generic function with 1 method)

In [42]:
t_val = 0.1
U_val::Float64 = 2
μ_val::Float64 = 0
sites::Int = 4

4

In [43]:
E, N = HubbardDMRG(t_val, U_val, μ_val, sites)

After sweep 1 energy=-0.08182451152062864  maxlinkdim=9 maxerr=0.00E+00 time=0.015


After sweep 2 energy=-0.08183179428408494  maxlinkdim=9 maxerr=0.00E+00 time=0.048


After sweep 3 energy=-0.08183179428408671  maxlinkdim=9 maxerr=0.00E+00 time=0.020


After sweep 4 energy=-0.08183179428408756  maxlinkdim=9 maxerr=0.00E+00 time=0.020


After sweep 5 energy=-0.08183179428408756  maxlinkdim=9 maxerr=0.00E+00 time=0.011


...

After sweep 8 energy=-0.08183179428408756  maxlinkdim=9 maxerr=0.00E+00 time=0.034


After sweep 9 energy=-0.08183179428408756  maxlinkdim=9 maxerr=0.00E+00 time=0.011


After sweep 10 energy=-0.08183179428408754  maxlinkdim=9 maxerr=0.00E+00 time=0.011


flux(psi) = QN("Number",4)


(-0.08183179428408754, [1.0000000000000002, 0.9999999999999999, 1.0000000000000002, 1.0000000000000002])